In [1]:
import subprocess
import yaml
import time

In [2]:
def ticcmd(*args):
    """ send the command to the controller
    
    Parameters:
        args -- tic command
    Precondition:
        the control mode of stepper controller is Serial/I2C/USB
    """ 
    return subprocess.check_output(['ticcmd'] + list(args))

def currentPos(ser):
    """ Get and print current position of the stepper

    Parameters:
        ser -- stepper controller id
    """
    status = yaml.load(ticcmd('-d',ser,'-s'))
    position = int(status['Current position'])
    print(position)
    
    return position

def setPos(ser,pos):
    """ Set the position of stepper
    
    Parameters:
        ser -- stepper controller id
        pos -- new position for this stepper
    """
    ticcmd('-d',ser,'--halt-and-set-position',str(pos))
    return

def setTarget(leftser,rightser,tar):
    """ Roll paper to the target
    
    Parameters:
        leftser -- left side stepper controller id
        rightser -- right side stepper controller id
        tar -- target of the paper roll, from -3000 to 3000
    Precondition:
        Set the current limit to 1092mA
        Set the stepper speed to 200, not sure, need check...
        Before the first time call this function, manually move the paper roll 
        to the original point and set two stepper's position to 0
    """
    t1 = time.time()
    t2 = time.time()
    
    #current position of one stepper
    pos = currentPos(leftser)
    
    #calculate the running time based on the speed
    runtime = abs(pos-tar)/190
    
    while ((t2-t1)<runtime):
        ticcmd('-d',leftser,'--exit-safe-start','--position', str(tar))
        ticcmd('-d',rightser,'--exit-safe-start', '--position', str(tar))
        t2 = time.time()
    
    #keep the tension of paper
    ticcmd('-d',leftser,'--exit-safe-start','--position', str(tar+45))
    ticcmd('-d',rightser,'--exit-safe-start','--position', str(tar-45))    
    time.sleep(0.5)
    
    #reset the position
    setPos(leftser,tar)
    setPos(rightser,tar)
    
    return

In [27]:
SERIAL1 = '00377013'
SERIAL2 = '00376994'

currentPos(SERIAL1)
currentPos(SERIAL2)

setTarget(SERIAL1,SERIAL2,0)

<ipython-input-2-823ee94a78bd>:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  status = yaml.load(ticcmd('-d',ser,'-s'))


0
0
0
